In [ ]:
import numpy as np
import scipy

from filterpy.monte_carlo import systematic_resample

from spf.dataset.spf_dataset import v5spfdataset

ds_fn = "/mnt/md1/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_10_03_38_21_nRX2_rx_circle.zarr"
ds_fn = "/mnt/md1/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_15_11_44_13_nRX2_bounce.zarr"
ds_fn = "/mnt/md1/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_03_00_30_00_nRX2_rx_circle.zarr"

import pickle
import os


output_prefix = "/tmp/" + os.path.basename(ds_fn) + "_"
# full_p_fn =
# full_p = pickle.load(open(full_p_fn, "rb"))["full_p"]

nthetas = 65
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache="/mnt/md2/cache/precompute_cache_3p5_chunk1",  # ="/home/mouse9911/precompute_cache_chunk16_fresh",
    paired=True,
    skip_fields=set(["signal_matrix"]),
    snapshots_per_session=-1,
    empirical_data_fn="/home/mouse9911/gits/spf/empirical_dists/full.pkl",
    empirical_individual_radio=False,
    empirical_symmetry=True,
)

# flip the order of the antennas
antenna_spacing = -ds.yaml_config["receivers"][0]["antenna-spacing-m"]
assert antenna_spacing == -ds.yaml_config["receivers"][1]["antenna-spacing-m"]

wavelength = ds.wavelengths[0]
assert wavelength == ds.wavelengths[1]

offsets = [
    ds.yaml_config["receivers"][0]["theta-in-pis"] * np.pi,
    ds.yaml_config["receivers"][1]["theta-in-pis"] * np.pi,
]

In [ ]:
pickle.load(open(full_p_fn, "rb"))["0.040000"]["r"].keys()

In [ ]:
from spf.model_training_and_inference.models.particle_filter import (
    PFSingleThetaSingleRadio,
)


pf = PFSingleThetaSingleRadio(ds=ds, rx_idx=1, full_p_fn="full_p.pkl")
trajectory, all_particles = pf.trajectory(
    mean=np.array([[0, 0]]), std=np.array([[2, 0.1]]), return_particles=True
)
pf.metrics(trajectory=trajectory)

In [ ]:
import matplotlib.pyplot as plt

from spf.rf import reduce_theta_to_positive_y

x = np.hstack(
    [np.repeat(idx, all_particles[idx].shape[0]) for idx in range(len(all_particles))]
)
y = np.hstack([all_particles[idx][:, 0] for idx in range(len(all_particles))])
plt.scatter(x, y, s=0.01, alpha=0.1)

plt.plot(
    reduce_theta_to_positive_y(ds[0][1]["ground_truth_theta"]),
    label=f"r1 reduced gt theta",
    color="black",
    linestyle="dashed",
    linewidth=1,
)

In [ ]:
import matplotlib.pyplot as plt

from spf.model_training_and_inference.models.particle_filter import (
    plot_single_theta_single_radio,
)

fig = plot_single_theta_single_radio(ds, full_p_fn)

fig.suptitle("Single ladies (radios) ParticleFilter")
fig.savefig(f"{output_prefix}_single_ladies_pf.png")

In [ ]:
from spf.model_training_and_inference.models.particle_filter import (
    PFSingleThetaDualRadio,
)
from spf.rf import pi_norm

pf = PFSingleThetaDualRadio(ds=ds, full_p_fn="full_p.pkl")

In [ ]:
traj_paired, _ = pf.trajectory(
    mean=np.array([[0, 0]]), std=np.array([[2, 0.1]]), return_particles=False
)

pf.metrics(trajectory=traj_paired)

In [ ]:
import numpy as np
from spf.model_training_and_inference.models.particle_filter import (
    plot_single_theta_dual_radio,
)

fig = plot_single_theta_dual_radio(ds=ds, full_p_fn=full_p_fn)
fig.suptitle("When two become one (Sensor fusion) Particle Filter")
fig.savefig(f"{output_prefix}_paired_pf.png")

In [ ]:
from spf.model_training_and_inference.models.particle_filter import PFXYDualRadio


pf = PFXYDualRadio(ds=ds, full_p_fn=full_p_fn)
traj_paired, _ = pf.trajectory(
    N=128 * 16,
    mean=np.array([[0, 0, 0, 0, 0]]),
    std=np.array([[0, 200, 200, 0.1, 0.1]]),
    return_particles=False,
    noise_std=np.array([[0, 15, 15, 0.5, 0.5]]),
)
pf.metrics(trajectory=traj_paired)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.model_training_and_inference.models.particle_filter import plot_xy_dual_radio

fig = plot_xy_dual_radio(ds, full_p_fn=full_p_fn)
fig.suptitle("When two become one (Sensor fusion) Particle Filter")
fig.savefig(f"{output_prefix}_paired_pf.png")